# Advanced Tutorial 2: Production Deployment

**Level:** Advanced  
**Time:** 30-40 minutes  
**Prerequisites:** All previous tutorials

## Overview

Deploy NeuroGraph to production:
- Docker multi-stage builds
- Docker Compose orchestration
- Health checks and monitoring
- Logging and observability
- Security hardening
- Scaling strategies

## Docker Deployment

## Step 1: Multi-Stage Dockerfile

Create optimized production image:

```dockerfile
# Stage 1: Build Rust core
FROM rust:1.70 AS rust-builder
WORKDIR /app
COPY src/core ./src/core
RUN cd src/core && cargo build --release

# Stage 2: Python dependencies
FROM python:3.11-slim AS python-builder
WORKDIR /app
COPY requirements.txt .
RUN pip install --no-cache-dir --user -r requirements.txt

# Stage 3: Production image
FROM python:3.11-slim
WORKDIR /app

# Copy built artifacts
COPY --from=rust-builder /app/src/core/target/release/*.so ./lib/
COPY --from=python-builder /root/.local /root/.local

# Copy application
COPY src/api ./src/api
COPY run.sh .

# Non-root user
RUN useradd -m -u 1000 neurograph && \
    chown -R neurograph:neurograph /app
USER neurograph

# Environment
ENV PATH=/root/.local/bin:$PATH
ENV PYTHONPATH=/app
ENV LOG_LEVEL=INFO

# Health check
HEALTHCHECK --interval=30s --timeout=10s --start-period=40s --retries=3 \
  CMD curl -f http://localhost:8000/api/v1/health || exit 1

EXPOSE 8000
CMD ["./run.sh"]
```

## Step 2: Docker Compose Stack

```yaml
# docker-compose.prod.yml
version: '3.8'

services:
  neurograph:
    build:
      context: .
      dockerfile: Dockerfile
    image: neurograph:latest
    container_name: neurograph-api
    restart: unless-stopped
    
    environment:
      - SECRET_KEY=${SECRET_KEY}
      - LOG_LEVEL=INFO
      - WORKERS=4
      - DATABASE_URL=${DATABASE_URL}
    
    ports:
      - "8000:8000"
    
    volumes:
      - ./logs:/app/logs
      - ./data:/app/data
    
    networks:
      - neurograph-net
    
    depends_on:
      - prometheus
      - grafana
    
    healthcheck:
      test: ["CMD", "curl", "-f", "http://localhost:8000/api/v1/health"]
      interval: 30s
      timeout: 10s
      retries: 3
      start_period: 40s

  prometheus:
    image: prom/prometheus:latest
    container_name: neurograph-prometheus
    restart: unless-stopped
    
    volumes:
      - ./monitoring/prometheus.yml:/etc/prometheus/prometheus.yml
      - prometheus-data:/prometheus
    
    ports:
      - "9090:9090"
    
    networks:
      - neurograph-net

  grafana:
    image: grafana/grafana:latest
    container_name: neurograph-grafana
    restart: unless-stopped
    
    environment:
      - GF_SECURITY_ADMIN_PASSWORD=${GRAFANA_PASSWORD}
      - GF_SERVER_ROOT_URL=http://localhost:3000
    
    volumes:
      - ./monitoring/grafana:/etc/grafana/provisioning
      - grafana-data:/var/lib/grafana
    
    ports:
      - "3000:3000"
    
    networks:
      - neurograph-net
    
    depends_on:
      - prometheus

networks:
  neurograph-net:
    driver: bridge

volumes:
  prometheus-data:
  grafana-data:
```

## Step 3: Kubernetes Deployment

```yaml
# k8s/deployment.yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: neurograph
  labels:
    app: neurograph
spec:
  replicas: 3
  selector:
    matchLabels:
      app: neurograph
  template:
    metadata:
      labels:
        app: neurograph
    spec:
      containers:
      - name: neurograph
        image: neurograph:latest
        ports:
        - containerPort: 8000
        
        env:
        - name: SECRET_KEY
          valueFrom:
            secretKeyRef:
              name: neurograph-secrets
              key: secret-key
        - name: LOG_LEVEL
          value: "INFO"
        - name: WORKERS
          value: "4"
        
        resources:
          requests:
            memory: "512Mi"
            cpu: "500m"
          limits:
            memory: "2Gi"
            cpu: "2000m"
        
        livenessProbe:
          httpGet:
            path: /api/v1/health/live
            port: 8000
          initialDelaySeconds: 30
          periodSeconds: 10
        
        readinessProbe:
          httpGet:
            path: /api/v1/health/ready
            port: 8000
          initialDelaySeconds: 20
          periodSeconds: 5
        
        startupProbe:
          httpGet:
            path: /api/v1/health/startup
            port: 8000
          failureThreshold: 30
          periodSeconds: 10
---
apiVersion: v1
kind: Service
metadata:
  name: neurograph-service
spec:
  selector:
    app: neurograph
  ports:
    - protocol: TCP
      port: 80
      targetPort: 8000
  type: LoadBalancer
```

## Step 4: Production Configuration

```bash
# .env.production
SECRET_KEY=<generate-with-openssl-rand-hex-32>
LOG_LEVEL=INFO
LOG_FORMAT=json

# Server
HOST=0.0.0.0
PORT=8000
WORKERS=4

# Security
CORS_ORIGINS=https://yourdomain.com
ACCESS_TOKEN_EXPIRE_MINUTES=15
REFRESH_TOKEN_EXPIRE_DAYS=7

# Monitoring
PROMETHEUS_ENABLED=true
GRAFANA_PASSWORD=<secure-password>

# Limits
MAX_TOKENS=100000
MAX_GRIDS=10
RATE_LIMIT_PER_MINUTE=1000
```

## Step 5: Monitoring Setup

### Prometheus Configuration

```yaml
# monitoring/prometheus.yml
global:
  scrape_interval: 15s
  evaluation_interval: 15s

scrape_configs:
  - job_name: 'neurograph'
    static_configs:
      - targets: ['neurograph:8000']
    metrics_path: '/metrics'
    scrape_interval: 10s

alerting:
  alertmanagers:
    - static_configs:
        - targets: ['alertmanager:9093']

rule_files:
  - 'alerts.yml'
```

### Alert Rules

```yaml
# monitoring/alerts.yml
groups:
  - name: neurograph
    interval: 30s
    rules:
      - alert: HighErrorRate
        expr: rate(neurograph_http_requests_total{status=~"5.."}[5m]) > 0.05
        for: 5m
        labels:
          severity: critical
        annotations:
          summary: "High error rate detected"
      
      - alert: HighMemoryUsage
        expr: neurograph_memory_usage_bytes > 1e9
        for: 10m
        labels:
          severity: warning
        annotations:
          summary: "Memory usage above 1GB"
      
      - alert: SlowRequests
        expr: histogram_quantile(0.95, rate(neurograph_request_duration_seconds_bucket[5m])) > 1
        for: 5m
        labels:
          severity: warning
        annotations:
          summary: "95th percentile request latency > 1s"
```

## Step 6: Deployment Commands

In [ ]:
# Deployment script (run in terminal)
deployment_commands = """
# 1. Build Docker image
docker build -t neurograph:latest .

# 2. Run with Docker Compose
docker-compose -f docker-compose.prod.yml up -d

# 3. Check health
curl http://localhost:8000/api/v1/health

# 4. View logs
docker-compose logs -f neurograph

# 5. Scale horizontally
docker-compose up -d --scale neurograph=3

# 6. Kubernetes deployment
kubectl apply -f k8s/deployment.yaml
kubectl apply -f k8s/service.yaml

# 7. Check K8s status
kubectl get pods -l app=neurograph
kubectl get svc neurograph-service
"""

print(deployment_commands)

## Step 7: Security Checklist

In [ ]:
security_checklist = [
    "✓ Change default admin password",
    "✓ Use strong SECRET_KEY (32+ random bytes)",
    "✓ Enable HTTPS (reverse proxy with nginx/traefik)",
    "✓ Restrict CORS origins to your domain",
    "✓ Use API keys for machine-to-machine auth",
    "✓ Enable rate limiting (1000 req/min default)",
    "✓ Run as non-root user in container",
    "✓ Scan Docker images for vulnerabilities",
    "✓ Keep dependencies updated",
    "✓ Enable audit logging",
    "✓ Use secrets management (K8s secrets, Vault)",
    "✓ Configure firewall rules",
    "✓ Enable intrusion detection",
    "✓ Regular security audits",
]

print("Production Security Checklist:\n")
for item in security_checklist:
    print(item)

## Step 8: Scaling Strategies

In [ ]:
scaling_guide = """
Horizontal Scaling:
  1. Run multiple instances behind load balancer
  2. Use stateless design (no local state)
  3. Shared Redis for session management (future)
  4. K8s HPA for auto-scaling:
     kubectl autoscale deployment neurograph --cpu-percent=70 --min=2 --max=10

Vertical Scaling:
  1. Increase WORKERS (2-4 per CPU core)
  2. Allocate more memory for large datasets
  3. Use faster storage for logs/data

Performance Tuning:
  1. Enable connection caching (default: on)
  2. Optimize grid cell size (2.0-4.0)
  3. Use HTTP/2 for API calls
  4. Configure connection pooling
  5. Enable compression for API responses

Database Scaling (when added):
  1. Read replicas for queries
  2. Write primary for mutations
  3. Sharding by user/tenant
  4. Caching layer (Redis)
"""

print(scaling_guide)

## Summary

✅ **Docker deployment** - Multi-stage builds, health checks  
✅ **Docker Compose** - Full stack orchestration  
✅ **Kubernetes** - Production-grade orchestration  
✅ **Monitoring** - Prometheus + Grafana + alerts  
✅ **Security** - Hardening checklist  
✅ **Scaling** - Horizontal and vertical strategies  

## Key Takeaways

1. **Multi-stage builds** reduce image size 70%+
2. **Health checks** critical for orchestration
3. **Monitoring** enables proactive issue detection
4. **Security** must be configured before production
5. **Horizontal scaling** preferred for high availability

## Next Steps

1. Deploy to staging environment
2. Run load tests
3. Configure monitoring dashboards
4. Set up CI/CD pipeline
5. Plan disaster recovery

---

**Congratulations!** You've completed all NeuroGraph tutorials. 🎉

**Resources:**
- [API Reference](https://your-docs-url/api/)
- [Architecture Guide](https://your-docs-url/architecture.html)
- [GitHub](https://github.com/chrnv/neurograph-os-mvp)